<a href="https://colab.research.google.com/github/Pengyu-gis/MyDeepLearing/blob/main/ImageSegmentation/water_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect GPU

In [ ]:
!nvidia-smi

## Loading the Dataset

In [4]:
import requests

# The URL of the dataset
url = ''
# Sending a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open a file with write-binary mode, which allows writing data to the file in binary format.
    # This is important for non-text files, such as a ZIP.
    with open("Track1.zip", "wb") as file:
        file.write(response.content)
    print("Download successful!")
else:
    print("Error downloading the file:", response.status_code)


Download successful!


In [ ]:
!unzip /content/Track1.zip -d /content/Water_Bodies_Dataset/

In [ ]:
!pip install rasterio

In [8]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
from PIL import Image
import glob
import rasterio
import numpy as np

In [9]:
def read_image_tif(path):
    with rasterio.open(path) as src:
        # Assuming the first three bands are RGB; adjust as necessary for your data
        image = src.read([1, 2, 3])  # Read only the first three bands
        # Convert from (C, H, W) to (H, W, C) for PIL compatibility
        image = np.moveaxis(image, 0, -1)
        # Ensure image is in uint8 format for PIL
        image = np.clip(image, 0, 255).astype(np.uint8)
    return image

class WaterBodiesDataset(Dataset):
    def __init__(self, images_folder, masks_folder, image_transform=None, mask_transform=None):
        self.images_folder = images_folder
        self.masks_folder = masks_folder
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.images = [img.split('/')[-1] for img in glob.glob(images_folder + "/*.tif")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_name = self.images[idx]
        image_path = os.path.join(self.images_folder, image_name)
        mask_path = os.path.join(self.masks_folder, image_name.replace('.tif', '.png'))

        image = read_image_tif(image_path)  # Assuming read_image_tif returns a numpy array
        image = Image.fromarray(image)

        mask = Image.open(mask_path).convert("L")

        if self.image_transform:
            image = self.image_transform(image)

        if self.mask_transform:
            mask = self.mask_transform(mask)
        else:
            mask = transforms.ToTensor()(mask)  # Default transform for masks

        return image, mask

# Define separate transforms for images and masks
image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transform = transforms.Compose([
    transforms.ToTensor(),  # Add more transforms as needed, but no normalization
])

# Initialize Dataset with separate transforms
train_dataset = WaterBodiesDataset(images_folder='/content/Water_Bodies_Dataset/Track1/train/images',
                                   masks_folder='/content/Water_Bodies_Dataset/Track1/train/labels',
                                   image_transform=image_transform,
                                   mask_transform=mask_transform)


In [10]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=(1, 1))  # Assuming binary segmentation
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:03<00:00, 64.6MB/s]


In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 7
for epoch in range(num_epochs):
    model.train()
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


# save the model

In [ ]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'path_to_save_model/model_state_dict.pth')

# Optionally, save the entire model
torch.save(model, 'path_to_save_model/entire_model.pth')


# Calculating the F1 Score


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Assuming you have a DataLoader for your validation set called `val_loader`
model.eval()  # Set the model to evaluation mode

# Containers for true labels and predictions
true_labels = []
predictions = []

with torch.no_grad():
    for images, masks in val_loader:
        images = images.to(device)
        output = model(images)['out']  # Adjust depending on your model's output
        output = torch.sigmoid(output)  # Apply sigmoid to get probabilities
        output = (output > 0.5).int()  # Threshold probabilities to get binary mask

        # Flatten the masks and outputs to compute F1 score per image
        true_labels.extend(masks.cpu().view(-1).numpy())
        predictions.extend(output.cpu().view(-1).numpy())

# Compute F1 score
f1 = f1_score(true_labels, predictions, average='binary')  # or 'macro' if multi-class segmentation
print(f"F1 Score: {f1}")
